In [6]:
import gluonbook as gb
from mxnet import autograd,gluon,init,nd
from mxnet.gluon import loss as gloss,nn

## Dataset

In [12]:
batch_size=256
train_iter, test_iter = gb.load_data_fashion_mnist(batch_size)

## Dropout

In [7]:
def dropout(X,drop_prob):
    mask=nd.random.uniform(0,1,X.shape)>drop_prob
    return mask*X

In [8]:
num_inputs,num_outputs,num_hiddens1,num_hiddens2=28*28,10,256,256
W1=nd.random.normal(scale=0.01,shape=(num_inputs,num_hiddens1))
b1=nd.zeros(num_hiddens1)
W2=nd.random.normal(scale=0.01,shape=(num_hiddens1,num_hiddens2))
b2=nd.zeros(num_hiddens2)
W3=nd.random.normal(scale=0.01,shape=(num_hiddens2,num_outputs))
b3=nd.zeros(num_outputs)
params=[W1,b1,W2,b2,W3,b3]
for param in params:
    param.attach_grad()

In [10]:
drop_prob1,drop_prob2=0.2,0.5
def net(X):
    X=X.reshape((-1,num_inputs))
    H1=(nd.dot(X,W1)+b1).relu()
    if autograd.is_training():
        H1=dropout(H1,drop_prob1)
    H2=(nd.dot(H1,W2)+b2).relu()
    if autograd.is_training():
        H2=dropout(H2,drop_prob2)
    return nd.dot(H2,W3)+b3

## train & eva

In [11]:
net=nn.Sequential()
net.add(nn.Dense(256,activation='relu'),
       nn.Dropout(drop_prob1),
       nn.Dense(256,activation='relu'),
       nn.Dropout(drop_prob2),
       nn.Dense(10))

In [13]:
lr,num_epochs=0.5,5
net.initialize(init.Normal(sigma=0.01))
loss=gloss.SoftmaxCrossEntropyLoss()
trainer=gluon.Trainer(net.collect_params(),'sgd',{'learning_rate':lr})
gb.train_ch3(net,train_iter,test_iter,loss,num_epochs,batch_size,None,None,trainer)

epoch 1, loss 1.1680, train acc 0.550, test acc 0.780
epoch 2, loss 0.5838, train acc 0.783, test acc 0.825
epoch 3, loss 0.5016, train acc 0.816, test acc 0.850
epoch 4, loss 0.4510, train acc 0.837, test acc 0.852
epoch 5, loss 0.4215, train acc 0.848, test acc 0.860
